In [ ]:
COLAB = False
#COLAB = True

In [ ]:
import mne
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from IPython.display import clear_output

In [ ]:
path="labeled_10mins_checkedSHA.edf"

In [ ]:
file = mne.io.read_raw_edf(path)

In [ ]:
raw_data = file.get_data()

In [ ]:
file

In [ ]:
plt.figure(figsize=(18, 5))
plt.plot(raw_data[2][1800000:1810000])

In [ ]:
def broad_filter(signal, fs, lowcut=0.1, highcut=35):
    """Returns filtered signal sampled at fs Hz, with a [lowcut, highcut] Hz
    bandpass."""
    # Generate butter bandpass of order 3.
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(3, (low, high), btype='band')
    # Apply filter to the signal with zero-phase.
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal

In [ ]:
plt.figure(figsize=(18, 5))
plt.plot(broad_filter(raw_data[4][1800000:1810000], 256))

In [ ]:
classes = np.load("labels.npy")

In [ ]:
classes

In [ ]:
plt.figure(figsize=(18, 5))
plt.plot(classes[0])

In [ ]:
path="labeled_10mins_checkedSHA.edf"

In [ ]:
sample_rate = 500
signal_size = signals.shape[1]
signal_duration = signal_size / sample_rate
print(f"Частота сигнала - {sample_rate} Гц")
print(f"Продолжительность сигнала - {signal_duration} сек.")

In [ ]:
# Вспомогательные функции, превращающие маску в сегментацию

v_to_del = {1:'p'}

def remove_small(signal):
    max_dist = 12
    last_zero = 0
    for i in range(len(signal)):
        if signal[i] == 0:
            if i - last_zero < max_dist:
                signal[last_zero:i] = 0
            last_zero = i

def merge_small(signal):
    max_dist = 12
    lasts = np.full(signal.max() + 1, -(max_dist+1))
    for i in range(len(signal)):
        m = signal[i]
        if i - lasts[m] < max_dist and m > 0:
            signal[lasts[m]:i] = m
        lasts[m] = i

def mask_to_delineation(mask):
    # merge_small(mask)
    # remove_small(mask)
    delineation = {'p':[]}
    i = 0
    mask_length = len(mask)
    while i < mask_length:
        v = mask[i]
        if v > 0:
            delineation[v_to_del[v]].append([i, 0])
            while i < mask_length and mask[i] == v:
                delineation[v_to_del[v]][-1][1] = i
                i += 1
            t = delineation[v_to_del[v]][-1]
        i += 1
    return delineation

In [ ]:
mask_to_delineation(get_mask(signals_test[i]))

In [ ]:
delineation = np.load("delineation.npy")

In [ ]:
delineation

In [ ]:
classes = np.zeros(300000)
for label in delineation:
    for tick in range(label[0], label[1]+1):
        classes[tick]=1

In [ ]:
sample_rate=500

In [ ]:
plt.rcParams['axes.linewidth'] = 5
plt.rcParams['xtick.major.width'] = 5
plt.rcParams['ytick.major.width'] = 5
plt.rcParams['xtick.major.size'] = 15
plt.rcParams['ytick.major.size'] = 15
def plot_signal_with_mask(signal, delineation, delineation2, i, k):
    fig, axs = plt.subplots(figsize=(18, 5), dpi=300)
    axs.spines[['right', 'top']].set_visible(False)
    #plt.title("Signal with mask")
    axs.xaxis.set_tick_params(labelsize=25)
    axs.yaxis.set_tick_params(labelsize=25)
    #plt.title("Signal with mask")
    plt.xlabel("Time (sec)", fontsize=25)
    plt.ylabel("Amplitude (μV)", fontsize=25)
    x_axis_values = np.linspace(0, len(signal) / sample_rate, len(signal))
    plt.plot(x_axis_values, signal, linewidth=4, color="black")
    
    # for begin, end in delineation['p']:
    #     begin /= sample_rate
    #     end /= sample_rate
    #     plt.axvspan(begin, end, facecolor='red', alpha=0.5)

    for begin, end in delineation2['p']:
        begin /= sample_rate
        end /= sample_rate
        plt.axvspan(begin, end, facecolor='green', alpha=0.5)
    plt.savefig(f"{i}_{k}_dert_nn_only.png", bbox_inches='tight')

    fig, axs = plt.subplots(figsize=(18, 5), dpi=300)
    axs.spines[['right', 'top']].set_visible(False)
    #plt.title("Signal with mask")
    axs.xaxis.set_tick_params(labelsize=25)
    axs.yaxis.set_tick_params(labelsize=25)
        #plt.title("Signal with mask")
    plt.xlabel("Time (sec)", fontsize=25)
    plt.ylabel("Amplitude (μV)", fontsize=25)
    x_axis_values = np.linspace(0, len(signal) / sample_rate, len(signal))
    plt.plot(x_axis_values, signal, linewidth=4, color="black")
    
    for begin, end in delineation['p']:
        begin /= sample_rate
        end /= sample_rate
        plt.axvspan(begin, end, facecolor='red', alpha=0.5)

    for begin, end in delineation2['p']:
        begin /= sample_rate
        end /= sample_rate
        plt.axvspan(begin, end, facecolor='green', alpha=0.5)
    plt.savefig(f"{i}_{k}_dert_nn_and_doc.png", bbox_inches='tight')

    fig, axs = plt.subplots(figsize=(18, 5), dpi=300)
    axs.spines[['right', 'top']].set_visible(False)
    #plt.title("Signal with mask")
    axs.xaxis.set_tick_params(labelsize=25)
    axs.yaxis.set_tick_params(labelsize=25)
    plt.xlabel("Time (sec)", fontsize=25)
    plt.ylabel("Amplitude (μV)", fontsize=25)
    x_axis_values = np.linspace(0, len(signal) / sample_rate, len(signal))
    plt.plot(x_axis_values, signal, linewidth=4, color="black")
    
    for begin, end in delineation['p']:
        begin /= sample_rate
        end /= sample_rate
        plt.axvspan(begin, end, facecolor='red', alpha=0.5)

    # for begin, end in delineation2['p']:
    #     begin /= sample_rate
    #     end /= sample_rate
    #     plt.axvspan(begin, end, facecolor='green', alpha=0.5)
    plt.savefig(f"{i}_{k}_dert_doc_only.png", bbox_inches='tight')

In [ ]:
plot_signal_with_mask(X_new[0], delineation[:9], 10, 1, 1)

In [ ]:
labels = np.load("labels.npy")

In [ ]:
labels[0]


In [ ]:
all_signals = raw_data[2][1800000:2100000]

In [ ]:
X=[]
Y=[]
for i in range(60):
    X.append(all_signals[5000*i:5000*i+5000])
    Y.append(classes[5000*i:5000*i+5000])

In [ ]:
X_new = np.load("X_new.npy")
Y_new = np.load("Y_new.npy")

In [ ]:
X_new_2 = np.load("X_new_2.npy")
Y_new_2 = np.load("Y_new_2.npy")

In [ ]:
X_new_3 = np.load("X_new_3.npy")
Y_new_3 = np.load("Y_new_3.npy")

In [ ]:
X_new_5 = np.load("1X.npy")
Y_new_5 = np.load("1Y.npy")

In [ ]:
X_new_6 = np.load("2X.npy")
Y_new_6 = np.load("2Y.npy")

In [ ]:
X = [X_new, X_new_2, X_new_3, X_new_5, X_new_6]
Y = [Y_new, Y_new_2, Y_new_3, Y_new_5, Y_new_6]
def get_cross_val(num):
    X_train=[]
    Y_train=[]
    X_test=[]
    Y_test=[]
    cnt=0
    for i in range(len(X)):
        if num==i:
            for k in range(len(X[i])):
                if max(Y[i][k])!=0.:
                    X_test.append(X[i][k])
                    Y_test.append(Y[i][k])
        else:
            for j in range(len(X[i])):
                if max(Y[i][j])!=0.:
                    cnt+=1
                    X_train.append(X[i][j])
                    Y_train.append(Y[i][j])
    print(cnt)
    return X_train, X_test, Y_train, Y_test

In [ ]:
plt.figure(figsize=(18, 5))
plt.plot(X_new_2[0])

In [ ]:
mask_to_delineation(Y[0])

In [ ]:
plot_signal_with_mask(X_new_2[31], mask_to_delineation(Y_new_2[31]))

In [ ]:
plt.figure(figsize=(18, 5))
plt.plot(Y[0])

In [ ]:
np.save("data.npy", X)
np.save("labels.npy", Y)

In [ ]:
for i in range(len(X_new)):
    X.append(X_new[i])
    Y.append(Y_new[i])


for i in range(len(X_new_2)):
    X.append(X_new_2[i])
    Y.append(Y_new_2[i])


# for i in range(len(X_new_3)):
#     X.append(X_new_3[i])
#     Y.append(Y_new_3[i])

In [ ]:
len(X)

In [ ]:
X_train = X
Y_train =Y
X_test = X_new_3
Y_test = Y_new_3

In [ ]:
# signals_train, signals_test, delins_train, delins_test= \
#     train_test_split(X, Y, train_size=0.8, test_size=0.2)
# X_train=[]
# Y_train=[]
# X_test=[]
# Y_test=[]
for i in range(4):
    X_train, X_test, Y_train, Y_test = get_cross_val(i)
    for i in range(len(X_train)):
        if (max(Y_train[i]) != 0.):
            X_train.append(signals_train[i])
            Y_train.append(delins_train[i])
    
    for i in range(len(X_test)):
        if (max(Y_test[i]) != 0.):
            X_test.append(signals_test[i])
            Y_test.append(delins_test[i])

In [ ]:
plt.plot(delins_test[12])

In [ ]:
len(Y_test)

In [ ]:
for i in range(len(signals_test)):
    print(len(signals_test[i]))
    print(len(delins_test[i]))

### Объявление Dataloader

In [ ]:
class SegmentationDataset(torch.utils.data.Dataset):
    def __init__(self, signals, masks):
        self._signals = torch.FloatTensor(np.expand_dims(signals, axis=1))
        self._masks = torch.LongTensor(masks)
        self._sub_len = 4000

    def __len__(self):
        return len(self._signals)

    def __getitem__(self, i):
        shift = np.random.randint(0, 1000)
        return self._signals[i, :, shift:shift+self._sub_len].clone().detach(), \
            self._masks[i, shift:shift+self._sub_len].clone().detach()

In [ ]:
torch.LongTensor(delins_train)[0]

In [ ]:
def get_loader(signals, masks, batch_size=4):
    dataset = SegmentationDataset(signals, masks)
    return DataLoader(dataset, batch_size=batch_size)

In [ ]:
train_loader = get_loader(X_train, Y_train)
val_loader = get_loader(X_test, Y_test)

In [ ]:
for x, y in train_loader:
    print(x)
    print("sfhesh")
    print(y)
    print("sghbds")

### Вспомогательные функции

In [ ]:
import torch.nn.functional as F

def get_loss(model, X_batch, y_batch):
    logits = model(X_batch)
    # logits = logits[:, :, 1000:-1000]
    # y_batch = y_batch[:, 1000:-1000]
    return F.cross_entropy(logits, y_batch).mean()

In [ ]:
class Drawer():
    def __init__(self):
        self._train = []
        self._val = []
    
    def add(self, train_loss, val_iou):
        self._train.append(train_loss)
        self._val.append(val_iou)
        
    def plot(self):
        epochs = range(1, 1 + len(self._train))
        plt.figure(figsize=(10, 8))
        plt.title('train/val loss')
        plt.xlabel('Эпоха')
        plt.ylabel('Loss')
        plt.plot(epochs, self._train)
        plt.plot(epochs, self._val)
        plt.legend(["train", "val"])
        plt.show()

In [ ]:
import time

def fit(model, train_loader, val_loader, optimizer, loss_function, num_epochs=None):
    train_losses = []
    val_losses = []
    drawer = Drawer()
    min_loss = 1e6
    min_loss_index = 0
    epoch = 0
    
    while num_epochs is None or epoch < num_epochs:
        start_time = time.time()
        model.train()
        for X_batch, y_batch in train_loader:
            loss = loss_function(model, X_batch, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_losses.append(loss.data.numpy())

        model.eval()
        for X_batch, y_batch in val_loader:
            loss = get_loss(model, X_batch, y_batch)
            val_losses.append(loss.data.numpy())
            
        train_loss = np.mean(train_losses[-len(train_loader):])
        val_loss = np.mean(val_losses[-len(val_loader):])
        print(len(train_loader), len(val_loader))
        drawer.add(train_loss, val_loss)
        
        clear_output(True)
        drawer.plot()
        print("Epoch {} took {:.3f}s".format(epoch + 1, time.time() - start_time))
        print("training loss: \t{:.6f}".format(train_loss))
        print("validation loss: \t{:.6f}".format(val_loss))
        
        if val_loss < min_loss - 1e-5:
            min_loss = val_loss
            min_loss_index = epoch
        
        epoch += 1

### Объявление модели

![img](https://raw.githubusercontent.com/MoskalenkoViktor/dl_ecg/master/nn_like_unet.png)
[ https://arxiv.org/pdf/2001.04689.pdf ]

![img](https://raw.githubusercontent.com/MoskalenkoViktor/dl_ecg/master/cnn.jpeg)
[ https://cs231n.github.io/convolutional-networks/ ]

In [ ]:
# Один блок свёртки
class UNetConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNetConv, self).__init__()
        self._model = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=9, padding=4),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(),
            nn.Conv1d(out_channels, out_channels, kernel_size=9, padding=4),
            nn.BatchNorm1d(out_channels),
            nn.ReLU()
        )
    
    def forward(self, X):
        return self._model(X)
    

class UNetDown(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNetDown, self).__init__()
        self._model = nn.Sequential(
            nn.MaxPool1d(2),
            UNetConv(in_channels, out_channels)
        )
    
    def forward(self, X):
        return self._model(X)
    

class UNetUp(nn.Module):
    def __init__(self, in_channels, in_channels_skip, out_channels):
        super(UNetUp, self).__init__()
        self._up = nn.ConvTranspose1d(in_channels, in_channels, kernel_size=8, stride=2, padding=3)
        self._model = UNetConv(in_channels + in_channels_skip, out_channels)
    
    def forward(self, X_skip, X):
        X = self._up(X)  
        diff = X_skip.size()[2] - X.size()[2]
        X = F.pad(X, (diff // 2, diff - diff // 2))  
        return self._model(torch.cat([X_skip, X], dim=1))

In [ ]:
class UNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(UNet, self).__init__()
        n = 2
        self._input = UNetConv(in_channels, n)
        self._down1 = UNetDown(n, 2*n)
        self._down2 = UNetDown(2*n, 4*n)
        self._down3 = UNetDown(4*n, 8*n)
        self._down4 = UNetDown(8*n, 16*n)
        self._up1 = UNetUp(16*n, 8*n, 8*n)
        self._up2 = UNetUp(8*n, 4*n, 4*n)
        self._up3 = UNetUp(4*n, 2*n, 2*n)
        self._up4 = UNetUp(2*n, n, n)
        self._output = nn.Conv1d(n, num_classes, kernel_size=1)
        
    def forward(self, X):
        x1 = self._input(X)
        x2 = self._down1(x1)
        x3 = self._down2(x2)
        x4 = self._down3(x3)
        x = self._down4(x4)
        x = self._up1(x4, x)
        x = self._up2(x3, x)
        x = self._up3(x2, x)
        x = self._up4(x1, x)
        return self._output(x)

### Обучение

In [ ]:
model = UNet(1, 2)
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
print(model)

In [ ]:
def dice_coef(mask1, mask2):
    intersect = np.sum(mask1*mask2)
    fsum = np.sum(mask1)
    ssum = np.sum(mask2)
    dice = (2 * intersect ) / (fsum + ssum)
    dice = np.mean(dice)
    dice = round(dice, 3) # for easy reading
    return dice   

In [ ]:
plt.rcParams['axes.linewidth'] = 5
plt.rcParams['xtick.major.width'] = 5
plt.rcParams['ytick.major.width'] = 5
plt.rcParams['xtick.major.size'] = 15
plt.rcParams['ytick.major.size'] = 15
def plot_signal_with_mask(signal, delineation, delineation2, i, k):
    fig, axs = plt.subplots(figsize=(18, 5), dpi=300)
    axs.spines[['right', 'top']].set_visible(False)
    #plt.title("Signal with mask")
    axs.xaxis.set_tick_params(labelsize=25)
    axs.yaxis.set_tick_params(labelsize=25)

    plt.xlabel("Time (sec)", fontsize=25)
    plt.ylabel("Amplitude (μV)", fontsize=25)
    x_axis_values = np.linspace(0, len(signal) / sample_rate, len(signal))
    plt.plot(x_axis_values, signal, linewidth=4, color="black")
    
    # for begin, end in delineation['p']:
    #     begin /= sample_rate
    #     end /= sample_rate
    #     plt.axvspan(begin, end, facecolor='red', alpha=0.5)

    #for begin, end in delineation2['p']:
        # begin /= sample_rate
        # end /= sample_rate
        # plt.axvspan(begin, end, facecolor='green', alpha=0.5)
    plt.savefig(f"1_{k}_empty.png", bbox_inches='tight')


In [ ]:
# dice_coefs=[]
# for i in range(5):
i=2
X_train, X_test, Y_train, Y_test = get_cross_val(i)

train_loader = get_loader(X_train, Y_train)
val_loader = get_loader(X_test, Y_test)

model = UNet(1, 2)
optimizer = torch.optim.Adam(model.parameters())
fit(model, train_loader, val_loader, optimizer, get_loss, num_epochs=70)
for k in range(len(X_test)):
    aa = Y_test[k]
    plot_signal_with_mask(X_test[k], mask_to_delineation(aa), mask_to_delineation(get_mask(X_test[k])), i, k)
for j in range(len(Y_test)):
    mask = get_mask(X_test[j])
    dice_coefs.append(dice_coef(Y_test[j], mask))

In [ ]:
#X_train, X_test, Y_train, Y_test = get_cross_val()
plot_signal_with_mask(X[1][37], [], Y_test, i, 37)

In [ ]:
for sig in X_train:
    for sig2 in X_test:
        cnt=[]
        for i in range(5000):
            cnt.append(sig[i]-sig2[i])
        if (max(cnt==min(cnt)) and max(cnt)==0):
            print(0)

In [ ]:
len(dice_coefs)

In [ ]:
len(X_new)

In [ ]:
len(X_new_2)

In [ ]:
len(X_new_3)

In [ ]:
new_dice_coefs=[]
for i in range(len(dice_coefs)):
    if i not in range(54+59, 54+59+60):
        new_dice_coefs.append(dice_coefs[i])

In [ ]:
np.mean(new_dice_coefs)

In [ ]:
segmentation_model = model

In [ ]:
np.mean(dice_coefs)

In [ ]:
np.save('dice_coefs.npy', dice_coefs)

In [ ]:
plt.hist(dice_coefs)

In [ ]:
import scipy

In [ ]:
len(dice_coefs)

In [ ]:
scipy.stats.ttest_1samp(dice_coefs, popmean=0.5)

In [ ]:
len(X_train)+len(X_test)

### Проверяем модель

In [ ]:
model = model.eval()

In [ ]:
# получение маски по сигналу
def get_mask(signal):
    signal = np.expand_dims(signal, axis=(0, 1))
    signal = torch.FloatTensor(signal)
    mask = model(signal)[0]
    mask = mask.max(axis=0)[1]
    mask[:500] = 0
    mask[-500:] = 0
    return mask.data.numpy()

In [ ]:
len(Y_test)

In [ ]:
X_new_5 = np.load("X_new_5.npy")
Y_new_5 = np.load("Y_new_5.npy")

In [ ]:
Y_test

In [ ]:
for i in range(len(X_test)):
    aa = Y_test[i]
    plot_signal_with_mask(X_test[i], mask_to_delineation(aa), mask_to_delineation(get_mask(X_test[i])), i)
    #plot_signal_with_mask(X_test[i], mask_to_delineation(get_mask(X_test[i])))

In [ ]:
for i in range(len(Y_train)):
    aa = Y_train[i]
    plot_signal_with_mask(X_train[i], mask_to_delineation(aa), mask_to_delineation(get_mask(X_train[i])))
    #plot_signal_with_mask(X_train[i], mask_to_delineation(get_mask(X_train[i])))

In [ ]:
len(X)

In [ ]:
signals_test[12]

In [ ]:
ad = []
for num in delins_test[i]:
    ad.append(int(num))

In [ ]:
mask_to_delineation(np.array(ad))

In [ ]:
def plot_test_sample(index):
    signal = signals_test[index]
    true_mask = delins_test[index]

    mask = get_mask(signal)
    #plt.plot(true_mask)
    plot_signal_with_mask(signal, mask_to_delineation(mask))
    plot_signal_with_mask(signal, mask_to_delineation(true_mask))

In [ ]:
plot_test_sample(12)

In [ ]:
plot_test_sample(20)

In [ ]:
plot_test_sample(25)

### Рецептивное поле (receptive field)

![img](https://raw.githubusercontent.com/MoskalenkoViktor/dl_ecg/master/receptive_field.png)
[ https://medium.com/mlreview/a-guide-to-receptive-field-arithmetic-for-convolutional-neural-networks-e0f514068807 ]

In [ ]:
print(f"Число параметров модели: {sum(p.numel() for p in model.parameters())}")

In [ ]:
# КОД: узнать рецептивное поле

## Сохранение моделей

In [ ]:
arrhythmia_model.eval()
torch.onnx.export(arrhythmia_model, torch.randn(1, 279), "arrhythmia_model.onnx")

In [ ]:
if COLAB:
    from google.colab import files
    files.download("arrhythmia_model.onnx")

## Диагноз по сигналу

In [ ]:
# КОД
model = None
train_loader = None
val_loader = None
optimizer = None
loss_diagnosis = None

In [ ]:
#fit(model, train_loader, val_loader, optimizer, loss_diagnosis)

## Диагноз по сигналу + сегментации

In [ ]:
# КОД
model = None
train_loader = None
val_loader = None
optimizer = None
loss_diagnosis = None

In [ ]:
#fit(model, train_loader, val_loader, optimizer, loss_diagnosis)

In [ ]:
torch.save(segmentation_model.state_dict(), "good_model.pt")

In [ ]:
torch.save(segmentation_model.state_dict(), "good_model_020824.pt")